In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/root/code_Bao/stock_price_4_fun/notebook


In [2]:
import requests
import re
import time
import random
import urllib3

urllib3.disable_warnings()

In [3]:
from src.summarize_text import *

In [4]:
speech_to_text = SpeechSummaryProcessor(audio_path='data/audio_ogg.ogg')
text = speech_to_text.generate_speech_to_text()

Device cuda:0


In [5]:
text

"1: I'll go home at half past eight to pack my things\n\n2: At 9 o'clock I go to bed to write my essay and at 10 o'clock I go to bed\n"

In [6]:
symbol = 'SSI'

In [7]:
news_scraper = NewsScraper()
news_list = news_scraper.search_stock_news(symbol=symbol, date_format='year')
news = news_scraper.take_text_from_link(news_url=news_list[0])

In [8]:
new_summarizer = NewsSummarizer()
sum_text = new_summarizer.summary_news(news= news)

In [9]:
sum_text

'   Ngân hàng Nhà nước quay trở lại phát hành tín phiếu trên kênh thị trường mở. \nKhối lượng phát hành lần đầu đạt 10.000 tỷ đồng trong 3 phiên đầu tiên. \nTrên thực tế, việc phát hành tín phiếu cũng có thể coi là tích cực, thay vì việc Ngân hàng Nhà nước lựa chọn bán ngoại tệ.\n\n    Năm 2022, nguyên nhân chính đến từ việc tăng trưởng tín dụng chạm trần từ giữa năm. \nTăng trưởng tín dụng tính đến ngày 15/9 chỉ ở mức 5,5% so với đầu năm. \nNgân hàng Nhà nước chọn phát hành tín phiếu là phương án bắt đầu từ năm 2023.\n'

In [10]:
watch_list=['GEX', 'VIX', 'HHV', 'BSR', 'SHS', 'PDR', 'CTR', 'HAH', 'VNINDEX', 'VND']

In [11]:
news_db = StockNewsDatabase()
# news_db.update_stock_news(watch_list)

In [12]:
news_db.extract_text_for_stock(stock_symbol='HAH')

('   Bà Trần Thị Thơm - Kiểm soát viên Công ty Cổ phần Vận tải và Xếp dỡ Hải An. \nCô đặt nhầm lệnh thành HAH. \nCô nói: “Đây là lần đầu tiên tôi mắc phải sai lầm này”.\n\n    Thời điểm bà Thơm bán 5.000 cổ phiếu diễn ra trước giai đoạn cổ phiếu HAH giảm giá nghiêm trọng. \nKể từ đó, mã này đã giảm xuống quanh mức 30.000 đồng/chiếc vào thời điểm cuối tháng, tức chỉ bằng khoảng 1/3 giá thị trường.\n',
 'https://vnexpress.net/mot-sep-hah-ban-chui-co-phieu-vi-mat-kem-4682401.html')

In [13]:
news_db.get_all_stocks()

['BSR', 'VND', 'HAH']